In [ ]:
import cv2
import numpy as np
import glob

# Load Calibration Parameters
def load_calibration(calibration_file):
    with np.load(calibration_file) as data:
        camera_matrix = data['camera_matrix']
        distortion_coeffs = data['dist_coeffs']
    print('Calibration data loaded')
    return camera_matrix, distortion_coeffs

# Undistort Image
def undistort_image(image, camera_matrix, dist_coeffs):
    h, w = image.shape[:2]
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(camera_matrix, dist_coeffs, (w, h), 1, (w, h))
    undistorted_img = cv2.undistort(image, camera_matrix, dist_coeffs, None, newcameramtx)
    print('Image undistorted')
    return undistorted_img

# Harris Corner Detection
def harris_corner_detection(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = np.float32(gray)
    dst = cv2.cornerHarris(gray, 2, 3, 0.04)
    dst = cv2.dilate(dst, None)
    image[dst > 0.01 * dst.max()] = [0, 0, 255]
    print('Harris corners detected')
    return image, dst

# Match Features Between Images
def match_features(image1, image2):
    gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(gray1, None)
    kp2, des2 = sift.detectAndCompute(gray2, None)

    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)

    matched_points1 = np.float32([kp1[m.queryIdx].pt for m in matches])
    matched_points2 = np.float32([kp2[m.trainIdx].pt for m in matches])

    print(f'Matched {len(matches)} features')
    return matched_points1, matched_points2

# Create Mosaic
def create_mosaic(images, camera_matrix, dist_coeffs):
    undistorted_images = [undistort_image(img, camera_matrix, dist_coeffs) for img in images]
    mosaic = undistorted_images[0]

    for i, img in enumerate(undistorted_images[1:], start=1):
        print(f'Stitching image {i} into the mosaic')
        
        mosaic_corners_img, mosaic_corners = harris_corner_detection(mosaic)
        image_corners_img, image_corners = harris_corner_detection(img)
        
        points1, points2 = match_features(mosaic, img)
        
        H, mask = cv2.findHomography(points1, points2, cv2.RANSAC)
        img_warp = cv2.warpPerspective(mosaic, H, (mosaic.shape[1] + img.shape[1], mosaic.shape[0]))

        alpha = 0.5
        beta = 1.0 - alpha
        blended_img = cv2.addWeighted(img_warp[:img.shape[0], :img.shape[1]], alpha, img, beta, 0.0)
        
        mosaic[:img.shape[0], :img.shape[1]] = blended_img
        print(f'Image {i} stitched')

    print('Mosaic created')
    return mosaic

# Main
if __name__ == "__main__":
    calibration_file = '/home/trevis/bwsi-uav/laboratory_2024/week_1_Hw/camera_calibration.npz'
    camera_matrix, dist_coeffs = load_calibration(calibration_file)

    image_folder = '/home/trevis/bwsi-uav/laboratory_2024/week_1_Hw/camera_calibration_photo_mosaic/International Village - 15 Percent Overlap/*.jpg'
    image_files = sorted(glob.glob(image_folder))
    images = [cv2.imread(file) for file in image_files]

    if not images:
        print("No images found in the specified folder.")
    else:
        print(f'{len(images)} images loaded for stitching.')
        mosaic_image = create_mosaic(images, camera_matrix, dist_coeffs)
        np.savez('/home/trevis/bwsi-uav/laboratory_2024/week_1_Hw/camera_calibration_photo_mosaic/mosaic', mosaic_image)

        cv2.imshow('Mosaic', mosaic_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        cv2.imwrite('/home/trevis/bwsi-uav/laboratory_2024/week_1_Hw/camera_calibration_photo_mosaic/mosaic_image.jpg', mosaic_image)
        print('Mosaic saved and displayed')


